In [1]:
import ibis
from ibis import _
import os
from osgeo import gdal
from cng.utils import *
from cng.h3 import *
# from cng.zonal import * 
con = ibis.duckdb.connect(extensions = ["spatial", "h3"])
install_h3()
set_secrets(con)

In [2]:
set_secrets(con, "", "", "s3.amazonaws.com", "overturemaps-us-west-2", 'vhost')
overture = con.read_parquet('s3://overturemaps-us-west-2/release/2024-11-13.0/theme=divisions/type=division_area/*', filename=True, hive_partitioning=1)

wkt = overture.filter(_.subtype=="country").filter(_.country == "US").select(_.geometry).execute().geometry[0].wkt



In [3]:
# smaller test case
ca = (overture
       .filter(_.country == "US", _.subtype == "region")
       .select('region', 'geometry')
       .filter(_.region == "US-CA")
       .execute()
      )

ca_wkt = ca.geometry[0].wkt

In [ ]:
import os # for vsis3
os.environ['AWS_VIRTUAL_HOSTING'] = 'FALSE'

from osgeo import gdal
gdal.DontUseExceptions()

#see `gdal.WarpOptions?` for details. Also see resampling options in warper for large data
# NOTE! dest given before input!

input_url = "/vsicurl/https://minio.carlboettiger.info/public-carbon/cogs/vulnerable_c_total_2018.tif"
dest = "/vsis3/public-carbon/carbon.xyz"
gdal.Warp(dest, input_url, dstSRS = 'EPSG:4326', cutlineWKT = wkt, cropToCutline = True)



Warning 1: the source raster dataset has a SRS, but the cutline features
not.  We assume that the cutline coordinates are expressed in the destination SRS.
If not, cutline results may be incorrect.


In [ ]:

(con
  .read_csv("s3://public-carbon/tmp-carbon.xyz", 
            delim = ' ', 
            columns = {'X': 'FLOAT', 'Y': 'FLOAT', 'Z': 'INTEGER'})
  .mutate(h8 = h3_latlng_to_cell_string(_.Y, _.X, 8))
  .rename(carbon = "Z")
  .mutate(carbon = ibis.ifelse(_.carbon == 65535, None, _.carbon)) 
  .to_parquet("s3://public-carbon/hex/us-vulnerable-total-carbon-2018-h8.parquet")
)



In [36]:
# test read
con.read_parquet("s3://public-carbon/hex/us-vulnerable-total-carbon-2018-h8.parquet").filter(_.carbon > 100).head().execute()


,X,Y,carbon,h8
0,-123.173286,42.006592,155,88281c40a1fffff
1,-123.170586,42.006592,159,88281c40a1fffff
2,-123.167892,42.006592,123,88281c40a1fffff
3,-123.165199,42.006592,170,88281c40a1fffff
4,-123.162506,42.006592,163,88281c40adfffff
